In [ ]:
# Cell 1: Import Libraries
import pandas as pd
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Embedding, LSTM, Dropout, BatchNormalization, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




In [ ]:
# Cell 2: Define Constants
IMG_SIZE = 128  # Size to which images will be resized
MAX_SEQUENCE_LENGTH = 50  # Max length of the recipes


In [ ]:
# Cell 3: Load and Preprocess Data
# Load the merged CSV file
data = pd.read_csv('merged_recipes_with_images.csv')

# Prepare a dictionary to hold images for each recipe
recipe_images = {}

# Iterate through the data to group images by recipes
for index, row in data.iterrows():
    recipe = row['Recipe']  # Adjust based on your CSV column name
    images = row['Image Paths'].split(', ')  # Update to use the correct column name

    if recipe not in recipe_images:
        recipe_images[recipe] = []
    recipe_images[recipe].extend(images)

# Prepare dataset for CNN
X_images = []
y_recipes = []

for recipe, images in recipe_images.items():
    for image_path in images:
        img = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = img_to_array(img) / 255.0  # Normalize the images
        X_images.append(img_array)
        y_recipes.append(recipe)

# Convert to numpy arrays
X_images = np.array(X_images)
y_recipes = np.array(y_recipes)

# Convert y_recipes to categorical labels
y_recipes_unique, y_recipes_encoded = np.unique(y_recipes, return_inverse=True)
num_classes = len(y_recipes_unique)


In [ ]:
# Cell 4: Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X_images, y_recipes_encoded, test_size=0.2, random_state=42)


In [ ]:
#! Cell 5: Build CNN Model initial 
# def create_cnn_model(input_shape, num_classes):
#     model = Sequential()
#     model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Flatten())
#     model.add(Dense(128, activation='relu'))
#     model.add(Dense(num_classes, activation='softmax'))
#     return model

#! rakesh dropout
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))  # Dropout after first max pooling

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))  # Dropout after second max pooling

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout before the final output layer
    
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

#!  CNN Model for Image Classification
# def create_cnn_model(input_shape, num_classes):
#     model = Sequential()
    
#     # First Convolutional Block
#     model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     # Second Convolutional Block
#     model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    
#     # Third Convolutional Block (Additional)
#     model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    
#     model.add(Flatten())
    
#     # Fully Connected Layers
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.5))  # Increased dropout for better generalization
    
#     model.add(Dense(num_classes, activation='softmax'))
    
#     return model

# Create the CNN model
cnn_model = create_cnn_model(X_train.shape[1:], num_classes)
print(cnn_model.summary())




In [ ]:
# Cell 6: Compile and Train the Model
cnn_model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64)


In [ ]:
# Cell 7: Plot Training History
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label='val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

import matplotlib.pyplot as plt

# Extract data from the history object
history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

# Create an epochs range
epochs = range(1, len(loss) + 1)

# Plot training and validation loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()



In [ ]:
# Cell 8: Save the Model
cnn_model.save('cnn_model_3.h5')


In [ ]:


# Load your data
data = pd.read_csv('merged_recipes_with_images.csv')

# Assuming the column with recipes is named 'Recipe'
recipes = data['Recipe'].values  # Make sure this column exists in your CSV

# Initialize the Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(recipes)

# Get the vocabulary size (+1 for padding token)
vocab_size = len(tokenizer.word_index) + 1  

# Set the maximum sequence length (can be adjusted based on your data)
max_sequence_length = 50  

# Convert recipes to sequences of integers
sequences = tokenizer.texts_to_sequences(recipes)

# Pad the sequences to ensure they all have the same length
sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Prepare the target variable (y_recipes_lstm)
# Assuming y_recipes_lstm should be the labels shifted by one word
# y_recipes_lstm should represent the next word to predict
# It will take the sequences and use all but the first element as the target
y_recipes_lstm = np.array([seq[1:] for seq in sequences])  # Target is the sequence shifted by one
y_recipes_lstm = np.eye(vocab_size)[y_recipes_lstm]  # Convert to one-hot encoding

# Reshape y_recipes_lstm to match the expected shape
# This ensures that the shape is (samples, timesteps, features)
y_recipes_lstm = y_recipes_lstm.reshape(-1, max_sequence_length - 1, vocab_size)  # Adjusting timesteps


In [ ]:
# Cell 10: Build LSTM Model initial model arch.
# def create_lstm_model(input_shape, num_classes):
#     model = Sequential()
#     model.add(Embedding(input_dim=num_classes, output_dim=128, input_length=input_shape[1]))
#     model.add(LSTM(128, return_sequences=False))
#     model.add(Dense(num_classes, activation='softmax'))
#     return model

#? rakesh dropout
# def create_lstm_model(input_shape, num_classes):
#     model = Sequential()
#     model.add(Embedding(input_dim=num_classes, output_dim=128, input_length=input_shape[1]))
#     model.add(LSTM(128, return_sequences=False))
#     model.add(Dropout(0.5))  # Dropout layer to reduce overfitting
#     model.add(Dense(num_classes, activation='softmax'))    
#     return model

#? improvement after rakesh
 
# def create_lstm_model(input_shape, vocab_size):
#     model = Sequential()
    
#     # Embedding Layer
#     model.add(Embedding(input_dim=vocab_size, output_dim=256, input_length=input_shape[1]))
    
#     # Bidirectional LSTM for Better Sequence Learning
#     model.add(Bidirectional(LSTM(128, return_sequences=True)))
#     model.add(Dropout(0.25))
    
#     # Another LSTM Layer for More Depth
#     model.add(LSTM(128, return_sequences=False))
#     model.add(Dropout(0.25))
    
#     # Fully Connected Layer
#     model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.5))
    
#     model.add(Dense(vocab_size, activation='softmax'))
    
#     return model
# # Create the LSTM model
# lstm_model = create_lstm_model(sequences.shape, len(tokenizer.word_index) + 1)

# print(lstm_model.summary())


In [ ]:
# Load the merged CSV file
data = pd.read_csv('merged_recipes_with_images.csv')

# Prepare a dictionary to hold images and recipes
recipe_images = {}
recipe_texts = {}

# Iterate through the data to group images by recipes and collect recipe texts
for index, row in data.iterrows():
    recipe = row['Recipe']  # Adjust based on your CSV column name
    images = row['Image Paths'].split(', ')  # Adjust based on your CSV column name
    recipe_text = row['Recipe']  # Assuming you have a column for recipe text

    # Store images
    if recipe not in recipe_images:
        recipe_images[recipe] = []
    recipe_images[recipe].extend(images)

    # Store recipe text
    recipe_texts[recipe] = recipe_text


In [ ]:
# Prepare the tokenizer for the recipe texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(recipe_texts.values())  # Fit tokenizer on all recipe texts

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(recipe_texts.values())

# Pad the sequences to ensure uniform input size
sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Convert the sequences to the format required for LSTM
# Use the previous sequences as input and the next word as the target
X_recipes = []
y_recipes_lstm = []

for seq in sequences:
    for i in range(1, len(seq)):
        X_recipes.append(seq[:i])  # Previous words
        y_recipes_lstm.append(seq[i])  # Next word

# Pad the sequences for LSTM input
X_recipes = pad_sequences(X_recipes, maxlen=MAX_SEQUENCE_LENGTH)

# Convert to numpy arrays
X_recipes = np.array(X_recipes)
y_recipes_lstm = np.array(y_recipes_lstm)

# Convert y_recipes_lstm to categorical labels
y_recipes_lstm = to_categorical(y_recipes_lstm, num_classes=len(tokenizer.word_index) + 1)


In [ ]:
# Build the LSTM model
#def create_lstm_model(input_shape, num_classes):
    #model = Sequential()
    #model.add(Embedding(input_dim=num_classes, output_dim=128, input_length=input_shape[1]))
    #model.add(LSTM(128, return_sequences=True))
   # model.add(LSTM(64))
  #  model.add(Dense(num_classes, activation='softmax'))
 #   return model

# Create the LSTM model
#lstm_model = create_lstm_model(X_recipes.shape, len(tokenizer.word_index) + 1)

# Compile the model
#lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model
#history_lstm = lstm_model.fit(X_recipes, y_recipes_lstm, validation_split=0.2, epochs=10, batch_size=32)


In [ ]:
from keras.layers import LSTM, Dropout

# def create_lstm_model(input_shape, num_classes):
#     model = Sequential()
#     model.add(Embedding(input_dim=10000, output_dim=128, input_length=input_shape[1]))  # Adjust parameters as needed
#     model.add(LSTM(128, return_sequences=True))
#     model.add(Dropout(0.25))  # Add dropout layer after LSTM
#     model.add(LSTM(64))  # You can adjust this layer as necessary
#     model.add(Dropout(0.5))  # Add another dropout layer
#     model.add(Dense(num_classes, activation='softmax'))
#     return model

#  #Create the LSTM model
# lstm_model = create_lstm_model(X_recipes.shape, len(tokenizer.word_index) + 1)

def create_lstm_model(input_shape, vocab_size):
    model = Sequential()
    
    # Embedding Layer
    model.add(Embedding(input_dim=vocab_size, output_dim=256, input_length=input_shape[1]))
    
    # Bidirectional LSTM for Better Sequence Learning
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.25))
    
    # Another LSTM Layer for More Depth
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.25))
    
    # Fully Connected Layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(vocab_size, activation='softmax'))
    
    return model
# Create the LSTM model
lstm_model = create_lstm_model(X_recipes.shape, len(tokenizer.word_index) + 1)
print(lstm_model.summary())
# lstm_model = create_lstm_model(sequences.shape, len(tokenizer.word_index) + 1)

# Compile the model
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model
history_lstm = lstm_model.fit(X_recipes, y_recipes_lstm, validation_split=0.2, epochs=50, batch_size=64)



In [ ]:
import matplotlib.pyplot as plt

# Extract data from history
history_dict = history_lstm.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

# Create epochs range
epochs = range(1, len(loss) + 1)

# Plot training and validation loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()


In [ ]:
# Save the LSTM model
lstm_model.save('lstm_model_3.h5')
